In [29]:
import torch
import torch.nn as nn

In [24]:
encoder = torch.nn.Linear(4, 17)
a = torch.ones([32, 64, 64, 4])
b = encoder(a)
b.shape



torch.Size([32, 64, 64, 17])

In [30]:
# from ccnn2.model_constructor import construct_models
from omegaconf import OmegaConf

cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ppo_trainer.yaml"
nn_cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/nn/nn.yaml"
ppo_cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ppo/ppo.yaml"
ccnn_cfg_img_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ccnn_img/ccnn_img.yaml"
ccnn_cfg_seq_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ccnn_seq/ccnn_seq.yaml"

cfg = OmegaConf.load(cfg_path)
nn_cfg = OmegaConf.load(nn_cfg_path)
ppo_cfg = OmegaConf.load(ppo_cfg_path)
ccnn_seq_cfg = OmegaConf.load(ccnn_cfg_img_path)
ccnn_img_cfg = OmegaConf.load(ccnn_cfg_seq_path)

cfg.nn = nn_cfg
cfg.ppo = ppo_cfg
cfg.ccnn_seq = ccnn_seq_cfg
cfg.ccnn_img = ccnn_img_cfg

cfg

# img_model, seq_model = construct_models(cfg)

{'defaults': [{'ppo': 'ppo'}, {'nn': 'nn'}, {'ccnn_img': 'ccnn_img'}, {'ccnn_seq': 'ccnn_seq'}, '_self_'], 'hydra': {'run': {'dir': 'outputs/${now:%Y-%m-%d/%H-%M-%S}'}}, 'experiment': {'env_ids': ['Breakout-v5'], 'seed': 42, 'max_episode_steps': 1000, 'num_rollout_steps': 128, 'num_envs': 64, 'total_timesteps': 10000000, 'save_ckpt': False, 'num_checkpoints': 20, 'print_interval': 100, 'stop_after_epochs': 500, 'capture_video': False, 'device': 2, 'cuda': True, 'torch_deterministic': True, 'resume': False, 'resume_update_idx': 0, 'resume_dir': 'None'}, 'evaluation': {'eval_seed': 3142, 'every': 8, 'num_eval': 5, 'num_test_envs': 5}, 'wandb': {'mode': 'online', 'project': 'DomainAgnosticRL', 'entity': None, 'name': None, 'group': None, 'tags': None, 'notes': None}, 'paths': {'dir': 'outputs/${now:%Y-%m-%d/%H-%M-%S}', 'log': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/runs', 'video': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/videos', 'checkpoints': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/checkpoints', 'src': 'o

In [31]:
class Resnet(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    # TODO
    kernel_size = cfg.nn.cnn.kernel_size
    stride = cfg.nn.cnn.stride
    in_channels = 4
    out_channels = 64
    final_channels = cfg.nn.actor_critic.d_model
    num_layers = cfg.nn.cnn.num_layers
    self.leakyrelu = nn.LeakyReLU()
    self.first_block = nn.Sequential(
          nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride,
                            padding=1, bias=True),
          nn.BatchNorm2d(out_channels),
          nn.LeakyReLU())
    
    self.conv_layers = nn.ModuleList()
    self.channels = [out_channels for i in range(num_layers)]
    for i in range(num_layers):  
      conv_block = nn.Sequential(
                                nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride,
                                padding=1, bias=True),
                                nn.LeakyReLU())
                                
      self.conv_layers.append(conv_block)
    self.final_block = nn.Sequential(
          nn.Conv2d(out_channels, final_channels, kernel_size=kernel_size, stride=stride,
                            padding=1, bias=True),
          nn.BatchNorm2d(out_channels),
          nn.LeakyReLU())
    self.avg_pooling = nn.AdaptiveAvgPool2d(output_size=1)

  def forward(self, x):
    if len(x.shape) < 4:
      x = x.unsqueeze(0)
    x = self.first_block(x)
    shortcut = x
    for conv_block in self.conv_layers:
      x = conv_block(x)
      x += shortcut
      shortcut = x
    x = self.avg_pooling(x)
    x = x.squeeze().squeeze()
    return x


In [32]:
resnet = Resnet(cfg)

In [33]:
total_params = 0
for name, param in resnet.named_parameters():
    total_params += param.numel()
total_params

150336

In [35]:
rand_img = torch.randn([32, 4, 64, 64])
out = resnet(rand_img)
out.shape

torch.Size([32, 64])

In [14]:
total_params = 0
for name, param in seq_model.named_parameters():
    total_params += param.numel()
total_params

233455

In [3]:
import torch

In [6]:
seq_input = torch.randn([32, 1, 1211])

out = seq_model(seq_input)
out.shape

/home/kukjin/.conda/envs/darl2/lib/python3.10/site-packages/torch/nn/functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input to dropout2d and assuming that channel-wise "


torch.Size([32, 256])

In [7]:
from ccnn2.ccnn_models import resnet as models

In [15]:
def construct_model_img(
    cfg: OmegaConf,
):
    """
    :param cfg: configuration file
    :return: An instance of torch.nn.Module
    """
    # Get parameters of model from task type
    data_dim = 2
    in_channels = 3
    out_channels = 256
    data_type = "image"

    # Get type of model from task type
    net_type = f"{cfg.ccnn_img.net.type}_{data_type}"

    # Overwrite data_dim in cfg.net
    # cfg.net.data_dim = data_dim

    # Print automatically derived model parameters.
    print(
        # f"Automatic Parameters:\n dataset = {cfg.dataset.name}, "
        f"net_name = {net_type},"
        f" data_dim = {data_dim}"
        f" in_channels = {in_channels},"
        f" out_chanels = {out_channels}."
    )
    if out_channels == 2:
        print(
            "The model will output one single channel. We use BCEWithLogitsLoss for training."
        )

    # Create and return model
    net_type = getattr(models, net_type)
    network = net_type(
        in_channels=in_channels,
        out_channels=out_channels if out_channels != 2 else 1,
        net_cfg=cfg.ccnn_img.net,
        kernel_cfg=cfg.ccnn_img.kernel,
        conv_cfg=cfg.ccnn_img.conv,
        mask_cfg=cfg.ccnn_img.mask,
    )

    return network

def construct_model_seq(
    cfg: OmegaConf,
):
    """
    :param cfg: configuration file
    :return: An instance of torch.nn.Module
    """
    # Get parameters of model from task type
    data_dim = 1
    in_channels = 1
    out_channels = 256
    data_type = "sequence"

    # Get type of model from task type
    net_type = f"{cfg.ccnn_seq.net.type}_{data_type}"

    # Overwrite data_dim in cfg.net
    # cfg.net.data_dim = data_dim

    # Print automatically derived model parameters.
    print(
        # f"Automatic Parameters:\n dataset = {cfg.dataset.name}, "
        f"net_name = {net_type},"
        f" data_dim = {data_dim}"
        f" in_channels = {in_channels},"
        f" out_chanels = {out_channels}."
    )
    if out_channels == 2:
        print(
            "The model will output one single channel. We use BCEWithLogitsLoss for training."
        )

    # Create and return model
    net_type = getattr(models, net_type)
    network = net_type(
        in_channels=in_channels,
        out_channels=out_channels if out_channels != 2 else 1,
        net_cfg=cfg.ccnn_seq.net,
        kernel_cfg=cfg.ccnn_seq.kernel,
        conv_cfg=cfg.ccnn_seq.conv,
        mask_cfg=cfg.ccnn_seq.mask,
    )

    return network

In [16]:
img_model = construct_model_img(cfg)
seq_model = construct_model_seq(cfg)


net_name = ResNet_image, data_dim = 2 in_channels = 3, out_chanels = 256.
Block 0/4
Block 1/4
Block 2/4
Block 3/4
net_name = ResNet_sequence, data_dim = 1 in_channels = 1, out_chanels = 256.
Block 0/4
Block 1/4
Block 2/4
Block 3/4


In [17]:
seq_input = torch.randn([32, 1, 1211])
out = seq_model(seq_input)
out.shape

/home/kukjin/.conda/envs/darl2/lib/python3.10/site-packages/torch/nn/functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input to dropout2d and assuming that channel-wise "


torch.Size([32, 256])

In [18]:
img_input = torch.randn([1, 3, 256, 256])
out = img_model(img_input)

AssertionError: Convolutional kernels must have odd dimensionality. Received torch.Size([1, 3, 33, 20])

In [19]:
img_input = torch.randn([1, 4, 256, 256])
out = img_model(img_input)

RuntimeError: shape '[-1, 4, 33, 20]' is invalid for input of size 1980

In [7]:
dropout2d = torch.nn.Dropout2d()

In [13]:
out = dropout2d(seq_input)

/home/kukjin/.conda/envs/darl2/lib/python3.10/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [14]:
out.shape

torch.Size([32, 1211])